# PROCAT Experiments

In this notebook, we run experiments on the 10K set of real catalogues from the PROCAT dataset.

##### Imports

In [1]:
import ast
import copy
import json
import numpy as np
import os
import pandas as pd
import pickle
import random
import time

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import Parameter
from torch.autograd import Variable

from matplotlib import pyplot as plt
from tqdm import tqdm

# sacred
from sacred import Experiment
from sacred.observers import MongoObserver

# text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from itertools import chain
import nltk  # had to also run nltk.download('punkt')

# visuals
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
# show full width tables
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [3]:
# custom utils
from utils import count_params, train_epochs
from data import test_model_custom, compare_solved_sort_unique, \
    get_single_kendall_tau, get_single_spearman_rho, get_batch_rank_correlation, get_batch_rank_correlation_and_perc_valid
from procat_utils import get_catalogs_with_full_info, show_predicted_catalog, show_correct_catalog, get_prediction_as_offers, print_predicted_catalog

In [4]:
# reload if saved changes in modules
%load_ext autoreload
%autoreload 2

## Config

In [5]:
config = dict()

In [6]:
# sacred experiment
config['ptr_mask'] = True
config['dataset_folder'] = 'PROCAT'
config['dataset_name'] = config['dataset_folder']
config['task'] = 'real_catalogs'
config['experiment_name'] = config['task'] + '_' + config['dataset_folder'] + '_mask_' + str(config['ptr_mask'])
config['db_name'] = 'sacred'
config['db_url'] = 'localhost:27017'

##### Model

In [7]:
# architecture
config['ptr_word_emb_dim'] = 32
config['ptr_offer_emb_dim'] = 64
config['ptr_hid_dim'] = 64
config['ptr_offer_rnn_layers'] = 2
config['ptr_catalog_rnn_layers'] = 2
config['ptr_dropout_catalogs'] = 0.05
config['ptr_dropout_offers'] = 0.05
config['ptr_bidir_catalogs'] = True
config['ptr_bidir_offers'] = False

# training
config['batch_size'] = 64
config['learning_rate'] =  0.0001
config['num_epochs'] = 5  # set this to 150 or 200 for actual results

# persisting
config['model_path'] = './model/model.pkb'

##### Dataset

In [8]:
# cv size
config['test_small_size'] = 300

# text preprocessing
config['vocab_max'] =  300000 # vocabulary cap (for tokens)
config['unknown_token'] = '?UNK?'
config['eos_token'] = '?EOS?'
config['pad_token'] = '?PAD?'
config['max_tokens'] = 30

# catalog preprocessing
config['page_break_token'] = '?PAGE_BREAK?'
config['page_break_priority'] = 0
config['max_offer_tokens_per_catalog'] = 200  # choose based on distributions, includes page-breaks
config['pad_not_real_offer'] = '?NOT_REAL_OFFER?'
config['pad_not_real_offer_priority'] = -1

##### Config paths

In [9]:
# input
config['path_in_df_offers_csv'] = './{}/offer_features.csv'.format(config['dataset_folder'])
config['path_in_df_sections_csv'] = './{}/section_features.csv'.format(config['dataset_folder'])
config['path_in_df_catalog_csv'] = './{}/catalog_features.csv'.format(config['dataset_folder'])

config['path_in_df_catalog_train_csv'] = './{}/catalog_train_set_features.csv'.format(config['dataset_folder'])
config['path_in_df_catalog_test_csv'] = './{}/catalog_test_set_features.csv'.format(config['dataset_folder'])

config['path_in_dictionary'] = './{}/dictionary.pickle'.format(config['dataset_folder'])
config['path_in_catalog_id_to_section_ids'] = './{}/catalog_to_sections.pickle'.format(config['dataset_folder'])

config['path_in_section_id_to_offer_ids'] = './{}/section_to_offers.pickle'.format(config['dataset_folder'])
config['path_in_section_id_to_offer_vectors'] = './{}/section_id_to_offer_vectors.pickle'.format(config['dataset_folder'])
config['path_in_section_id_to_section_num'] = './{}/section_to_number.pickle'.format(config['dataset_folder'])
config['path_in_section_id_to_offer_priorities'] = './{}/section_id_to_offer_priorities.pickle'.format(config['dataset_folder'])

config['path_in_offer_id_to_priority'] = './{}/offer_to_priority.pickle'.format(config['dataset_folder'])
config['path_in_offer_id_to_vector'] = './{}/offer_to_vector.pickle'.format(config['dataset_folder'])

config['path_in_np_X_train'] = './{}/X_train.npy'.format(config['dataset_folder'])
config['path_in_np_Y_train'] = './{}/Y_train.npy'.format(config['dataset_folder'])
config['path_in_np_X_test'] = './{}/X_test.npy'.format(config['dataset_folder'])
config['path_in_np_Y_test'] = './{}/Y_test.npy'.format(config['dataset_folder'])

config['path_in_torch_X_train'] = './{}/X_train.pb'.format(config['dataset_folder'])
config['path_in_torch_Y_train'] = './{}/Y_train.pb'.format(config['dataset_folder'])
config['path_in_torch_X_test'] = './{}/X_test.pb'.format(config['dataset_folder'])
config['path_in_torch_Y_test'] = './{}/Y_test.pb'.format(config['dataset_folder'])

##### Training and Model

In [10]:
config

{'ptr_mask': True,
 'dataset_folder': 'PROCAT',
 'dataset_name': 'PROCAT',
 'task': 'real_catalogs',
 'experiment_name': 'real_catalogs_PROCAT_mask_True',
 'db_name': 'sacred',
 'db_url': 'localhost:27017',
 'ptr_word_emb_dim': 32,
 'ptr_offer_emb_dim': 64,
 'ptr_hid_dim': 64,
 'ptr_offer_rnn_layers': 2,
 'ptr_catalog_rnn_layers': 2,
 'ptr_dropout_catalogs': 0.05,
 'ptr_dropout_offers': 0.05,
 'ptr_bidir_catalogs': True,
 'ptr_bidir_offers': False,
 'batch_size': 64,
 'learning_rate': 0.0001,
 'num_epochs': 5,
 'model_path': './model/model.pkb',
 'test_small_size': 300,
 'vocab_max': 300000,
 'unknown_token': '?UNK?',
 'eos_token': '?EOS?',
 'pad_token': '?PAD?',
 'max_tokens': 30,
 'page_break_token': '?PAGE_BREAK?',
 'page_break_priority': 0,
 'max_offer_tokens_per_catalog': 200,
 'pad_not_real_offer': '?NOT_REAL_OFFER?',
 'pad_not_real_offer_priority': -1,
 'path_in_df_offers_csv': './PROCAT/offer_features.csv',
 'path_in_df_sections_csv': './PROCAT/section_features.csv',
 'path_in_

## Experiment Tracking via Sacred

In [11]:
# initiate the experiment
ex = Experiment(name=config['experiment_name'], interactive=True)

# add an observer, storing experiment info
ex.observers.append(MongoObserver(url=config['db_url'], db_name=config['db_name']))

In [12]:
# experiment config
@ex.config
def ex_config():
    
    # general
    learning_rate = config['learning_rate']
    dataset = config['dataset_name'] 
    epochs = config['num_epochs']
    cfg = config
    
    # model
    model = None
    final_training_loss = None
    optimizer = None
    
    # db 
    db_name = config['db_name']
    db_url = config['db_url']

In [13]:
# experiment run function
@ex.main
def run_model_tests():

    # run all tests, get all results
    result_general, result_tau, result_spearman, rank_valid_perc = run_tests(current_tested_model, test_dataloader, config)
    
    # log params and results
    log_experiment_results(result_general, result_tau, result_spearman, rank_valid_perc)
    
    return round(result_general, 5)

In [14]:
def run_tests(a_model, a_dataloader, a_config):
    
    print('Model: ', a_model.__class__)

    a_model.eval() 
    
    # general accuracy
    result_general, _ = test_model_custom(a_model, a_dataloader, compare_solved_sort_unique, print_every=999999, x_name=0, y_name=1)
    print('Result: {:.4f}'.format(result_general))
    
    result_tau, rank_valid_perc = get_batch_rank_correlation_and_perc_valid(a_dataloader, a_model, get_single_kendall_tau, print_every=999999)
    print('K-Tau: {:.4f}, perc_valid: {}'.format(result_tau, rank_valid_perc))

    result_spearman, rank_valid_perc = get_batch_rank_correlation_and_perc_valid(a_dataloader, a_model, get_single_spearman_rho, print_every=999999)
    print('S-Rho: {:.4f}, perc_valid: {}'.format(result_spearman, rank_valid_perc))

    a_model.train()
    
    return result_general, result_tau, result_spearman, rank_valid_perc

In [15]:
def log_experiment_results(r_general, r_tau, r_spearman, rank_valid_perc):
    
    # round if not none
    if r_tau:
        r_tau = round(r_tau, 5)
    if r_spearman:
        r_spearman = round(r_spearman, 5)
    
    # log the results
    ex.log_scalar('test.general', r_general)
    ex.log_scalar('test.rank_correlation_valid_perc', rank_valid_perc)
    ex.log_scalar('test.tau', r_tau, 5)
    ex.log_scalar('test.rho', r_spearman, 5)

### Custom utils

In [16]:
def recreate_unshuffled_x(x, y):
    r = [x[i] for i in y]
    return r

In [17]:
random.seed(220788)

In [18]:
def get_example(a_dataloader, x_name=0, y_name=1):
    """
    Take a torch dataloader and get a single example.
    """
    a_batch = next(iter(a_dataloader))
    example_points = a_batch[x_name][0]
    example_solution = a_batch[y_name][0]

    return example_points, example_solution


def get_batch(a_dataloader, x_name=0, y_name=1):
    """
    Get a batch of points and solutions from a torch dataloader.
    """
    a_batch = next(iter(a_dataloader))
    batch_points = a_batch[x_name]
    batch_solutions = a_batch[y_name]

    return batch_points, batch_solutions


# Data
Load all needed data.

## Reload Data
Including:
- **SIGIR csv's**
    - df_offers
    - df_sections
    - df_catalogs
- **any meta-resources**:
    - (word2idx)
    - section to offer ids etc?
- **numpy X and Y**
    - train and test separately
- **torch dataloaders**
    - the actual torch dataloaders (for myself?

#### Load CSVs

In [19]:
df = pd.read_csv(config['path_in_df_offers_csv'], sep=';')
df.head(1)

,catalog_id,section,offer_id,priority,heading,description,text,text_tokenized,token_length,offer_as_vector
0,0003leq,1,fed3AqfY,2,Påskebryg,"33 cl. KYLLE - KYLLE. 3 33. + PANT. Sælges fra fredag d. 17/3. Ved køb af mere end 60 stk., er prisen herefter 4,98 kr","Påskebryg 33 cl. KYLLE - KYLLE. 3 33. + PANT. Sælges fra fredag d. 17/3. Ved køb af mere end 60 stk., er prisen herefter 4,98 kr","['påskebryg', '33', 'cl', '.', 'kylle', '-', 'kylle', '.', '3', '33', '.', '+', 'pant', '.', 'sælges', 'fra', 'fredag', 'd.', '17/3', '.', 'ved', 'køb', 'af', 'mere', 'end', '60', 'stk.', ',', 'er', '?EOS?']",33,"[10912, 250, 50, 5, 16874, 18, 16874, 5, 35, 250, 5, 82, 128, 5, 267, 46, 2199, 147, 41453, 5, 93, 210, 22, 182, 227, 106, 3845, 6, 31, 0]"


In [20]:
len(df)

1613686

In [21]:
df_sections = pd.read_csv(config['path_in_df_sections_csv'], sep=';')
df_sections.head(1)

,catalog_id,section_id,section_num,offer_ids,offer_vectors,offer_priorities
0,0003leq,0003leq_1,1,"['fed3AqfY', '799dzqfY', 'b8a3DqfY', 'a4c88qfY', '3cf8qqfY', '88a5UqfY', '6d6e0qfY', 'ef59YqfY', 'ab38CqfY', '0cbffqfY', '0a4eEqfY']","[[10912, 250, 50, 5, 16874, 18, 16874, 5, 35, 250, 5, 82, 128, 5, 267, 46, 2199, 147, 41453, 5, 93, 210, 22, 182, 227, 106, 3845, 6, 31, 0], [408, 99, 1024, 15, 5, 29, 5, 37, 5, 136, 141, 46, 5612, 13, 1763, 7, 3228, 7, 4673, 7, 6008, 7, 6644, 0, 1, 1, 1, 1, 1, 1], [50781, 17910, 5, 967, 21, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1163, 712, 1063, 3313, 2145, 3686, 27, 5, 7, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1362, 16, 3494, 4462, 34761, 21, 5, 19, 20, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1585, 26725, 5, 19, 20, 66, 21, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [932, 11768, 21, 5, 7, 140, 7, 420, 5, 1787, 5, 168, 104, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [3772, 3048, 65, 27, 5, 7, 6182, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [882, 248, 7, 578, 5, 79, 21, 5, 93, 210, 22, 182, 227, 57, 425, 6, 31, 200, 202, 7012, 102, 5, 75, 31, 336, 160, 0, 1, 1, 1], [1587, 3925, 21, 5, 18, 152, 18, 465, 5, 688, 336, 5, 7, 727, 7, 800, 7, 1761, 2026, 7, 189, 9, 518, 7, 2168, 7, 272, 1004, 7, 0], [2359, 93, 210, 22, 182, 227, 65, 425, 6, 31, 200, 202, 11439, 102, 5, 112, 21, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]","[2, 2, 1, 1, 2, 1, 1, 1, 3, 3, 2]"


In [22]:
len(df_sections)

238256

In [23]:
df_catalogs = pd.read_csv(config['path_in_df_catalog_csv'], sep=';')
df_catalogs.head(1)

catalog_id                                                                                  section_ids                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             offer_ids_with_pb                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [24]:
len(df_catalogs)

11063

In [25]:
# reload feature csv's split into train and test
df_catalogs_train = pd.read_csv(config['path_in_df_catalog_train_csv'], sep=';')
# df_catalogs_train.head(1)

In [26]:
df_catalogs_test = pd.read_csv(config['path_in_df_catalog_test_csv'], sep=';')
# df_catalogs_test.head(1)

In [27]:
len(df_catalogs_train), len(df_catalogs_test)

(8850, 2212)

#### Load Meta Resources (dicts)

In [28]:
# read word2idx dictionary
with open(config['path_in_dictionary'], 'rb') as f:
    word2idx = pickle.load(f)
list(word2idx.items())[:5]

[('?EOS?', 0),
 ('?PAD?', 1),
 ('?UNK?', 2),
 ('?PAGE_BREAK?', 3),
 ('?NOT_REAL_OFFER?', 4)]

In [29]:
# read catalog id to section ids
with open(config['path_in_catalog_id_to_section_ids'], 'rb') as f:
    catalog_id_to_section_ids = pickle.load(f)
print(list(catalog_id_to_section_ids.items())[:2])

[('0003leq', {'0003leq_5', '0003leq_3', '0003leq_4', '0003leq_1', '0003leq_6', '0003leq_2', '0003leq_7'}), ('0007YiY', {'0007YiY_5', '0007YiY_14', '0007YiY_1', '0007YiY_11', '0007YiY_12', '0007YiY_16', '0007YiY_3', '0007YiY_2', '0007YiY_9', '0007YiY_15', '0007YiY_6', '0007YiY_7', '0007YiY_10', '0007YiY_13', '0007YiY_4', '0007YiY_8'})]


In [30]:
# read section maps
with open(config['path_in_section_id_to_offer_ids'], 'rb') as f:
    section_id_to_offer_ids = pickle.load(f)
print(list(section_id_to_offer_ids.items())[:2])

[('0003leq_1', ['fed3AqfY', '799dzqfY', 'b8a3DqfY', 'a4c88qfY', '3cf8qqfY', '88a5UqfY', '6d6e0qfY', 'ef59YqfY', 'ab38CqfY', '0cbffqfY', '0a4eEqfY']), ('0003leq_2', ['0ef3QqfY', 'a3abpqfY', 'ce94wqfY', 'e545VqfY', '11aaxqfY', 'e4e12qfY', '431baqfY', '3030ZqfY', 'db3fNqfY', 'd638dqfY', '539csqfY', 'b25flqfY', '18cdFqfY', '50b5vqfY'])]


In [31]:
with open(config['path_in_section_id_to_offer_vectors'], 'rb') as f:
    section_id_to_offers_as_vectors = pickle.load(f)
print(list(section_id_to_offers_as_vectors.items())[0])

('0003leq_1', [[10912, 250, 50, 5, 16874, 18, 16874, 5, 35, 250, 5, 82, 128, 5, 267, 46, 2199, 147, 41453, 5, 93, 210, 22, 182, 227, 106, 3845, 6, 31, 0], [408, 99, 1024, 15, 5, 29, 5, 37, 5, 136, 141, 46, 5612, 13, 1763, 7, 3228, 7, 4673, 7, 6008, 7, 6644, 0, 1, 1, 1, 1, 1, 1], [50781, 17910, 5, 967, 21, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1163, 712, 1063, 3313, 2145, 3686, 27, 5, 7, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1362, 16, 3494, 4462, 34761, 21, 5, 19, 20, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1585, 26725, 5, 19, 20, 66, 21, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [932, 11768, 21, 5, 7, 140, 7, 420, 5, 1787, 5, 168, 104, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [3772, 3048, 65, 27, 5, 7, 6182, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [882, 248, 7, 578, 5, 79, 21, 5, 93, 210, 22, 182, 227, 57, 425, 6, 31, 200, 202, 

In [32]:
with open(config['path_in_section_id_to_section_num'], 'rb') as f:
    section_id_to_section_num = pickle.load(f)
print(list(section_id_to_section_num.items())[:5])

[('0003leq_1', 1), ('0003leq_2', 2), ('0003leq_3', 3), ('0003leq_4', 4), ('0003leq_5', 5)]


In [33]:
with open(config['path_in_section_id_to_offer_priorities'], 'rb') as f:
    section_ids_to_offers_as_priorities = pickle.load(f)
print(list(section_ids_to_offers_as_priorities.items())[:2])

[('0003leq_1', [2, 2, 1, 1, 2, 1, 1, 1, 3, 3, 2]), ('0003leq_2', [3, 3, 1, 2, 1, 1, 2, 1, 3, 3, 1, 1, 1, 1])]


In [34]:
# read offer maps
with open(config['path_in_offer_id_to_priority'], 'rb') as f:
    offer_id_to_priority = pickle.load(f)
print(list(offer_id_to_priority.items())[:2])

[('fed3AqfY', 2), ('799dzqfY', 2)]


In [35]:
with open(config['path_in_offer_id_to_vector'], 'rb') as f:
    offer_id_to_vector = pickle.load(f)
print(list(offer_id_to_vector.items())[:2])

[('fed3AqfY', [10912, 250, 50, 5, 16874, 18, 16874, 5, 35, 250, 5, 82, 128, 5, 267, 46, 2199, 147, 41453, 5, 93, 210, 22, 182, 227, 106, 3845, 6, 31, 0]), ('799dzqfY', [408, 99, 1024, 15, 5, 29, 5, 37, 5, 136, 141, 46, 5612, 13, 1763, 7, 3228, 7, 4673, 7, 6008, 7, 6644, 0, 1, 1, 1, 1, 1, 1])]


#### Load Numpy Matrices (X & Y)

In [36]:
# train set
X_train = np.load(config['path_in_np_X_train'])
Y_train = np.load(config['path_in_np_Y_train'])
X_train.shape, Y_train.shape, X_train[0], Y_train[0]

((8850, 200, 30),
 (8850, 200),
 array([[    4,     4,     4, ...,     4,     4,     4],
        [    4,     4,     4, ...,     4,     4,     4],
        [    4,     4,     4, ...,     4,     4,     4],
        ...,
        [    4,     4,     4, ...,     4,     4,     4],
        [18785,   118,    21, ...,     1,     1,     1],
        [ 5353, 20580,    38, ...,     1,     1,     1]], dtype=int32),
 array([ 14,  71,  78,  75, 116,  16,  79, 121, 148, 101,  49, 174,  13,
         53,  62, 155,  65, 159, 167,   6,  94, 114,  18, 120,  32,  12,
        166, 150,  44, 165, 183,  63, 198,   8,  98,  23,  84, 151, 192,
        163,  72,  15,  47, 189, 129,  99,  97,  73,  34, 117, 125, 135,
         38,  19,  95, 164,  83,  92, 102,  37, 170, 199, 122, 188, 112,
        186, 157, 138, 195, 184, 179, 132, 145, 144, 181,  36, 141, 178,
         20, 142, 115, 175,  77, 130,  24, 128,  21, 113,  70,  27, 152,
         29,  96,  93, 173,  60,  42,  86, 194,  30,  39,  68,  57, 131,
          5, 1

In [37]:
# test set
X_test = np.load(config['path_in_np_X_test'])
Y_test = np.load(config['path_in_np_Y_test'])
X_test.shape, Y_test.shape, X_test[0], Y_test[0]

((2212, 200, 30),
 (2212, 200),
 array([[    4,     4,     4, ...,     4,     4,     4],
        [ 7090,    68,    21, ...,     1,     1,     1],
        [ 3487, 10083,     7, ...,     1,     1,     1],
        ...,
        [ 5873,  4054,  1274, ...,     1,     1,     1],
        [    4,     4,     4, ...,     4,     4,     4],
        [  552,  2849,  2494, ...,     1,     1,     1]], dtype=int32),
 array([169,  22,  51,  88,  63,  42, 152,   7, 188,  66, 196, 111, 109,
        155, 133, 162,  61, 117, 142, 192, 105, 166, 137, 181,   1,  21,
        106, 160,  43,  73,  40, 178, 146, 173,   2, 161,  45, 129, 197,
        112, 123, 118, 104, 145,  38, 186, 195,  56,   3,  24,  41,  78,
         86, 141,  54,  28, 115, 150, 187, 107,  33, 148,  60,  25,  70,
        135, 134,   5,   9,  10,  59,  46, 154,  72, 132, 127,  18,  92,
         31, 103,  67,  36,  87, 126,  71,  50,  65, 124, 171,   6, 114,
        102,  27, 138,  32, 101, 185,  53,  95,  16, 147,  99,  94, 139,
        158, 1

#### Load Torch Tensors (X & Y)

In [38]:
# train set
X_train_torch = torch.load(config['path_in_torch_X_train'])
Y_train_torch = torch.load(config['path_in_torch_Y_train'])
X_train_torch.size(), Y_train_torch.size()

(torch.Size([8850, 200, 30]), torch.Size([8850, 200]))

In [39]:
# full test set
X_test_full_torch = torch.load(config['path_in_torch_X_test'])
Y_test_full_torch = torch.load(config['path_in_torch_Y_test'])
X_test_full_torch.size(), Y_test_full_torch.size()

(torch.Size([2212, 200, 30]), torch.Size([2212, 200]))

In [40]:
# Y needs to be a long
Y_train_torch = Y_train_torch.long()
Y_test_full_torch = Y_test_full_torch.long()

In [41]:
Y_train_torch.type(), Y_test_full_torch.type()

('torch.LongTensor', 'torch.LongTensor')

#### Smaller test set
For validation here, due to memory size problems.

In [42]:
config['test_small_size']

300

In [43]:
# split
X_test_torch = X_test_full_torch[:config['test_small_size']]
Y_test_torch = Y_test_full_torch[:config['test_small_size']]
X_test_torch.size(), Y_test_torch.size()

(torch.Size([300, 200, 30]), torch.Size([300, 200]))

#### Turn into Torch Datasets and Dataloaders

In [44]:
# into datasets
dataset_train = TensorDataset(X_train_torch, Y_train_torch)
dataset_test = TensorDataset(X_test_torch, Y_test_torch)

In [45]:
# dataloaders
train_dataloader = DataLoader(dataset_train, batch_size=config['batch_size'], shuffle=True, num_workers=4)
test_dataloader = DataLoader(dataset_test, batch_size=config['batch_size'], shuffle=True, num_workers=4)

# Models

## Model PtrNet

In [46]:
from procat_models import PointerCatalogNetwork, \
    PointerEncoder, PointerAttention, PointerDecoder, PointerOfferEmbedder

In [47]:
# instantiate the model
model_ptrnet = PointerCatalogNetwork(
    word_embedding_dim=config['ptr_word_emb_dim'],
    offer_embedding_dim=config['ptr_offer_emb_dim'],
    hidden_dim=config['ptr_hid_dim'],
    offer_rnn_layers=config['ptr_offer_rnn_layers'],
    catalog_rnn_layers=config['ptr_catalog_rnn_layers'],
    dropout_offers=config['ptr_dropout_offers'],
    dropout_catalogs=config['ptr_dropout_catalogs'],
    bidir_offers=config['ptr_bidir_offers'],  # not handled 2021 04
    bidir_catalogs=config['ptr_bidir_catalogs'],
    masking=config['ptr_mask'],
    vocab_size=len(word2idx),
)

model_ptrnet.float()

PointerCatalogNetwork(
  (word_embedding): Embedding(300005, 32)
  (offer_embedding): PointerOfferEmbedder(
    (offer_rnn): GRU(32, 64, num_layers=2, batch_first=True, dropout=0.05)
  )
  (encoder): PointerEncoder(
    (lstm): LSTM(64, 32, num_layers=2, batch_first=True, dropout=0.05, bidirectional=True)
  )
  (decoder): PointerDecoder(
    (input_to_hidden): Linear(in_features=64, out_features=256, bias=True)
    (hidden_to_hidden): Linear(in_features=64, out_features=256, bias=True)
    (hidden_out): Linear(in_features=128, out_features=64, bias=True)
    (att): PointerAttention(
      (input_linear): Linear(in_features=64, out_features=64, bias=True)
      (context_linear): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (soft): Softmax(dim=1)
    )
  )
)

In [48]:
# how many params
count_params(model_ptrnet)

The model has 9,744,032 trainable parameters


In [49]:
# 2-elem batch
batch_x = X_train_torch[:2]

In [50]:
# predict
_ = model_ptrnet(batch_x)
print(_[0].size(), _[1].size())

torch.Size([2, 200, 200]) torch.Size([2, 200])


In [51]:
print(_[1].type())

torch.LongTensor


In [52]:
# loss
CCE = torch.nn.CrossEntropyLoss()

In [53]:
# optimizer
optimizer_ptrnet = optim.Adam(
    filter(lambda p: p.requires_grad, model_ptrnet.parameters()),
    lr=config['learning_rate'])

### Model Training | PtrNet

In [54]:
# train
last_loss_ptrnet = train_epochs(
    model_ptrnet,
    optimizer_ptrnet,
    CCE,
    train_dataloader,
    config['num_epochs'],
    allow_gpu=True,
    x_name=0,
    y_name=1)

CUDA available, using GPU:
Quadro P6000


Epoch 5 / 5: 100%|██████████| 139/139 [02:47<00:00,  1.20s/ batches, avg_loss=5.29432]


In [55]:
last_loss_ptrnet.item()

5.294321060180664

### Model Testing | PtrNet
Put the model in eval() mode first (prevents dropout and such from predicting sth else from the same permuted x).

In [56]:
model_ptrnet.eval()  # if we had dropout or batchnorm, we must do this

PointerCatalogNetwork(
  (word_embedding): Embedding(300005, 32)
  (offer_embedding): PointerOfferEmbedder(
    (offer_rnn): GRU(32, 64, num_layers=2, batch_first=True, dropout=0.05)
  )
  (encoder): PointerEncoder(
    (lstm): LSTM(64, 32, num_layers=2, batch_first=True, dropout=0.05, bidirectional=True)
  )
  (decoder): PointerDecoder(
    (input_to_hidden): Linear(in_features=64, out_features=256, bias=True)
    (hidden_to_hidden): Linear(in_features=64, out_features=256, bias=True)
    (hidden_out): Linear(in_features=128, out_features=64, bias=True)
    (att): PointerAttention(
      (input_linear): Linear(in_features=64, out_features=64, bias=True)
      (context_linear): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (soft): Softmax(dim=1)
    )
  )
)

#### Sacred Experiment | Model PtrNet
We'll want to track the dataset, model params and the result of tests.

In [57]:
# experiment config
current_tested_model = model_ptrnet
current_optimizer = optimizer_ptrnet
current_last_loss = last_loss_ptrnet

config_update = {
    'model': current_tested_model.__class__.__name__,
    'final_training_loss': round(current_last_loss.item(), 10),
    'optimizer': current_optimizer.__class__.__name__,
    'cfg': config,
}

# run the experiment, with updated config
run_info = ex.run(config_updates=config_update)

INFO - real_catalogs_PROCAT_mask_True - Running command 'run_model_tests'
INFO - real_catalogs_PROCAT_mask_True - Started run with ID "90"


Model:  <class 'procat_models.PointerCatalogNetwork'>
Result: 0.0091
K-Tau: 0.2968, perc_valid: 100.0


INFO - real_catalogs_PROCAT_mask_True - Result: 0.00908
INFO - real_catalogs_PROCAT_mask_True - Completed after 0:02:01


S-Rho: 0.4414, perc_valid: 100.0


#### Show predictions
See what it predicts.

##### Specific catalog prediction, with offer text features

In [58]:
# choose a catalog
chosen_catalog_id = '0003leq'

In [59]:
# show correct
show_correct_catalog(
    catalog_id=chosen_catalog_id,
    catalogs_dataframe=df_catalogs,
    offers_dataframe=df)

	fed3AqfY h: Påskebryg 33 cl. KYLLE - KYLLE. 3 33. + , d: 33 cl. KYLLE - KYLLE. 3 33. + PANT. Sælg
	799dzqfY h: Frisk dansk mælk PR. LITER. Max. 24 time, d: PR. LITER. Max. 24 timer fra gård til bu
	b8a3DqfY h: Sukkervafler BELGISKE. 550 gram, d: BELGISKE. 550 gram
	a4c88qfY h: Hatting fransk hotdog  gigant burgerboll, d: 4-6 stk. •
	3cf8qqfY h: Oreo eller prince chokoladekiks 154-240 , d: 154-240 gram. Flere varianter
	88a5UqfY h: Knækbrød LEKSANDS. Flere varianter 200 g, d: LEKSANDS. Flere varianter 200 gram
	6d6e0qfY h: Carletti 73,5 gram. • Lys • Mørk. PÅLÆGS, d: 73,5 gram. • Lys • Mørk. PÅLÆGSCHOKOLADE
	ef59YqfY h: Corny müslibars 6 stk. • Nuts, d: 6 stk. • Nuts
	ab38CqfY h: BKI KAFFE  • Extra. 400 gram. Ved køb af, d: • Extra. 400 gram. Ved køb af mere end 1
	0cbffqfY h: Pålækker 80-90 gram. - Classic - Tomat. , d: 80-90 gram. - Classic - Tomat. VILDT BIL
	0a4eEqfY h: Havregryn Ved køb af mere end 6 poser, e, d: Ved køb af mere end 6 poser, er prisen h
------------------------
	0

Use premade function to nicely display a single prediction.

**Models that do not properly ensure permutation invariance will give different prediction on each run** regardless of being put in .eval() mode.

In [60]:
# show predicted
# if the model is not permutation invariant, it will predict sth else
# for every random permutation of the source catalog
model_ptrnet.eval()
show_predicted_catalog(
    catalog_id='0003leq',
    a_model=model_ptrnet,
    catalogs_dataframe=df_catalogs,
    offers_dataframe=df)

	4809E0fY h: Haribo POSE. DET ER BILLIGT. 100-135 gra, d: POSE. DET ER BILLIGT. 100-135 gram. • Sl
	ce94wqfY h: Ice cap rejer 125 gram, d: 125 gram
	0ba1i0fY h: Bamseline skyllemiddel Flere varianter 1, d: Flere varianter 1000 ml.
	97f9K0fY h: Kylling I ovnklar stegepose 1500 gram. H, d: I ovnklar stegepose 1500 gram. HEL DANSK
	f4f4MqfY h: Tulip PAKKE. 200 gram. MIDDAGS- FRIKADEL, d: PAKKE. 200 gram. MIDDAGS- FRIKADELLER
	b553q0fY h: Helt kalkunbryst Ca. 800 gram, d: Ca. 800 gram
	ef59YqfY h: Corny müslibars 6 stk. • Nuts, d: 6 stk. • Nuts
	9381iqfY h: Tortilla chips SANTA MARIA. 185 gram. • , d: SANTA MARIA. 185 gram. • Salted • Cheese
	aeberqfY h: Yoggi yoghurt 1000 gram. . Flere variant, d: 1000 gram. . Flere varianter
	614ebqfY h: Hellmann's mayonnaise • I glas • I tube., d: • I glas • I tube. 225-400 gram
	44a2IqfY h: Tulip •. 300-350 gram. • Baconpostej • K, d: •. 300-350 gram. • Baconpostej • Krydder
	59cchqfY h: Flødehavarti 300 gram. VILDT BILLIGT, d: 300 gram. VILDT BILLIGT


## Model DeepSets

In [61]:
from procat_models import DeepSets, DeepSetsPointerCatalogNetwork

In [62]:
# instantiate the model
model_deepsets = DeepSetsPointerCatalogNetwork(
    word_embedding_dim=config['ptr_word_emb_dim'],
    offer_embedding_dim=config['ptr_offer_emb_dim'],
    hidden_dim=config['ptr_hid_dim'],
    offer_rnn_layers=config['ptr_offer_rnn_layers'],
    catalog_rnn_layers=config['ptr_catalog_rnn_layers'],
    dropout_offers=config['ptr_dropout_offers'],
    dropout_catalogs=config['ptr_dropout_catalogs'],
    bidir_offers=config['ptr_bidir_offers'],  # not handled 2021 04
    bidir_catalogs=config['ptr_bidir_catalogs'],
    masking=config['ptr_mask'],
    vocab_size=len(word2idx),
)

model_deepsets.float()

DeepSetsPointerCatalogNetwork(
  (word_embedding): Embedding(300005, 32)
  (offer_embedding): PointerOfferEmbedder(
    (offer_rnn): GRU(32, 64, num_layers=2, batch_first=True, dropout=0.05)
  )
  (set_embedding): DeepSets(
    (encode): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ReLU()
      (4): Linear(in_features=64, out_features=64, bias=True)
      (5): ReLU()
      (6): Linear(in_features=64, out_features=128, bias=True)
      (7): ReLU()
      (8): Linear(in_features=128, out_features=128, bias=True)
      (9): ReLU()
      (10): Linear(in_features=128, out_features=128, bias=True)
      (11): ReLU()
      (12): Linear(in_features=128, out_features=64, bias=True)
      (13): ReLU()
    )
    (out): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=128, bias=True)
      (3

In [63]:
# how many params
count_params(model_deepsets)

The model has 9,863,776 trainable parameters


In [64]:
# 2-elem batch
batch_x = X_train_torch[:2]

In [65]:
# predict
_ = model_deepsets(batch_x)
print(_[0].size(), _[1].size())

torch.Size([2, 200, 200]) torch.Size([2, 200])


In [66]:
print(_[1].type())

torch.LongTensor


In [67]:
# loss
CCE = torch.nn.CrossEntropyLoss()

In [68]:
# optimizer
optimizer_deepsets = optim.Adam(
    filter(lambda p: p.requires_grad, model_deepsets.parameters()),
    lr=config['learning_rate'])

### Model Training | DeepSets

In [69]:
# train
last_loss_deepsets = train_epochs(
    model_deepsets,
    optimizer_deepsets,
    CCE,
    train_dataloader,
    config['num_epochs'],
    allow_gpu=True,
    x_name=0,
    y_name=1)

CUDA available, using GPU:
Quadro P6000


Epoch 5 / 5: 100%|██████████| 139/139 [03:05<00:00,  1.33s/ batches, avg_loss=5.29586]


In [70]:
last_loss_deepsets.item()

5.295864105224609

### Model Testing | DeepSets
Put the model in eval() mode first (prevents dropout and such from predicting sth else from the same permuted x).

In [71]:
model_deepsets.eval()  # if we had dropout or batchnorm, we must do this

DeepSetsPointerCatalogNetwork(
  (word_embedding): Embedding(300005, 32)
  (offer_embedding): PointerOfferEmbedder(
    (offer_rnn): GRU(32, 64, num_layers=2, batch_first=True, dropout=0.05)
  )
  (set_embedding): DeepSets(
    (encode): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ReLU()
      (4): Linear(in_features=64, out_features=64, bias=True)
      (5): ReLU()
      (6): Linear(in_features=64, out_features=128, bias=True)
      (7): ReLU()
      (8): Linear(in_features=128, out_features=128, bias=True)
      (9): ReLU()
      (10): Linear(in_features=128, out_features=128, bias=True)
      (11): ReLU()
      (12): Linear(in_features=128, out_features=64, bias=True)
      (13): ReLU()
    )
    (out): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=128, bias=True)
      (3

#### Sacred Experiment | Model DeepSets
We'll want to track the dataset, model params and the result of tests.

In [72]:
# experiment config
current_tested_model = model_deepsets
current_optimizer = optimizer_deepsets
current_last_loss = last_loss_deepsets

config_update = {
    'model': current_tested_model.__class__.__name__,
    'final_training_loss': round(current_last_loss.item(), 10),
    'optimizer': current_optimizer.__class__.__name__,
    'cfg': config,
}

# run the experiment, with updated config
run_info = ex.run(config_updates=config_update)

INFO - real_catalogs_PROCAT_mask_True - Running command 'run_model_tests'
INFO - real_catalogs_PROCAT_mask_True - Started run with ID "91"


Model:  <class 'procat_models.DeepSetsPointerCatalogNetwork'>
Result: 0.0092
K-Tau: 0.2857, perc_valid: 100.0


INFO - real_catalogs_PROCAT_mask_True - Result: 0.00918
INFO - real_catalogs_PROCAT_mask_True - Completed after 0:03:14


S-Rho: 0.4263, perc_valid: 100.0


#### Show predictions
See what it predicts.

##### Specific catalog prediction, with offer text features

In [73]:
# choose a catalog
chosen_catalog_id = '0003leq'

Use premade function to nicely display a single prediction.

In [74]:
# show predicted
# if the model is not permutation invariant, it will predict sth else
# for every random permutation of the source catalog
show_predicted_catalog(
    catalog_id='0003leq',
    a_model=model_deepsets,
    catalogs_dataframe=df_catalogs,
    offers_dataframe=df)

	88a5UqfY h: Knækbrød LEKSANDS. Flere varianter 200 g, d: LEKSANDS. Flere varianter 200 gram
	d8f9oqfY h: Margarine •. 500 gram. BINE. BILLIGST PÅ, d: •. 500 gram. BINE. BILLIGST PÅ HELE INDK
	5b2ag0fY h: Amanda luksusrogn Pr, d: Pr
	029aT0fY h: Abena affaldsposer PR. SAM-PAK. 3 x, d: PR. SAM-PAK. 3 x
	907ajqfY h: Æbler 2 kg. pink lady. vildt billigt, d: 2 kg. pink lady. vildt billigt
	2502x0fY h: Leeuwenkuil TA' • Chenin Blanc • Shiraz , d: TA' • Chenin Blanc • Shiraz • Lion's Lai
	b7fcb0fY h: Harpic Wc-blåt, d: 
	799dzqfY h: Frisk dansk mælk PR. LITER. Max. 24 time, d: PR. LITER. Max. 24 timer fra gård til bu
	a367w0fY h: Libresse Flere varianter. • Trusseindlæg, d: Flere varianter. • Trusseindlæg • Bind
	22edWqfY h: Flensted kartoffelgratin med fløde 500 g, d: med fløde 500 gram.
	9381iqfY h: Tortilla chips SANTA MARIA. 185 gram. • , d: SANTA MARIA. 185 gram. • Salted • Cheese
	4809E0fY h: Haribo POSE. DET ER BILLIGT. 100-135 gra, d: POSE. DET ER BILLIGT. 100-135 gram. • Sl
	b8a3Dqf

	a259L0fY h: Sild i spand 800/400-700 gram. • Mariner, d: 800/400-700 gram. • Marinerede hele file


## Model SetTransformer

In [75]:
from procat_models import SetTransformer, SetTransformerPointerCatalogNetwork

In [76]:
# instantiate the model
model_settrans = SetTransformerPointerCatalogNetwork(
    word_embedding_dim=config['ptr_word_emb_dim'],
    offer_embedding_dim=config['ptr_offer_emb_dim'],
    hidden_dim=config['ptr_hid_dim'],
    offer_rnn_layers=config['ptr_offer_rnn_layers'],
    catalog_rnn_layers=config['ptr_catalog_rnn_layers'],
    dropout_offers=config['ptr_dropout_offers'],
    dropout_catalogs=config['ptr_dropout_catalogs'],
    bidir_offers=config['ptr_bidir_offers'],  # not handled 2021 04
    bidir_catalogs=config['ptr_bidir_catalogs'],
    masking=config['ptr_mask'],
    vocab_size=len(word2idx),
)

model_settrans.float()

SetTransformerPointerCatalogNetwork(
  (word_embedding): Embedding(300005, 32)
  (offer_embedding): PointerOfferEmbedder(
    (offer_rnn): GRU(32, 64, num_layers=2, batch_first=True, dropout=0.05)
  )
  (set_embedding): SetTransformer(
    (emb): Sequential(
      (0): SAB(
        (mab): MAB(
          (fc_q): Linear(in_features=64, out_features=64, bias=True)
          (fc_k): Linear(in_features=64, out_features=64, bias=True)
          (fc_v): Linear(in_features=64, out_features=64, bias=True)
          (fc_o): Linear(in_features=64, out_features=64, bias=True)
        )
      )
      (1): SAB(
        (mab): MAB(
          (fc_q): Linear(in_features=64, out_features=64, bias=True)
          (fc_k): Linear(in_features=64, out_features=64, bias=True)
          (fc_v): Linear(in_features=64, out_features=64, bias=True)
          (fc_o): Linear(in_features=64, out_features=64, bias=True)
        )
      )
    )
    (enc): Sequential(
      (0): PMA(
        (mab): MAB(
          (fc_q)

In [77]:
# how many params
count_params(model_settrans)

The model has 9,864,224 trainable parameters


In [78]:
# 2-elem batch
batch_x = X_train_torch[:2]

In [79]:
# predict
_ = model_settrans(batch_x)
print(_[0].size(), _[1].size())

torch.Size([2, 200, 200]) torch.Size([2, 200])


In [80]:
print(_[1].type())

torch.LongTensor


In [81]:
# loss
CCE = torch.nn.CrossEntropyLoss()

In [82]:
# optimizer
optimizer_settrans = optim.Adam(
    filter(lambda p: p.requires_grad, model_settrans.parameters()),
    lr=config['learning_rate'])

### Model Training | Set Transformer

In [83]:
# train
last_loss_settrans = train_epochs(
    model_settrans,
    optimizer_settrans,
    CCE,
    train_dataloader,
    config['num_epochs'],
    allow_gpu=True,
    x_name=0,
    y_name=1)

CUDA available, using GPU:
Quadro P6000


Epoch 5 / 5: 100%|██████████| 139/139 [03:12<00:00,  1.38s/ batches, avg_loss=5.29431]


In [84]:
last_loss_settrans.item()

5.294308185577393

### Model Testing | Set Transformer
Put the model in eval() mode first (prevents dropout and such from predicting sth else from the same permuted x).

In [85]:
model_settrans.eval()  # if we had dropout or batchnorm, we must do this

SetTransformerPointerCatalogNetwork(
  (word_embedding): Embedding(300005, 32)
  (offer_embedding): PointerOfferEmbedder(
    (offer_rnn): GRU(32, 64, num_layers=2, batch_first=True, dropout=0.05)
  )
  (set_embedding): SetTransformer(
    (emb): Sequential(
      (0): SAB(
        (mab): MAB(
          (fc_q): Linear(in_features=64, out_features=64, bias=True)
          (fc_k): Linear(in_features=64, out_features=64, bias=True)
          (fc_v): Linear(in_features=64, out_features=64, bias=True)
          (fc_o): Linear(in_features=64, out_features=64, bias=True)
        )
      )
      (1): SAB(
        (mab): MAB(
          (fc_q): Linear(in_features=64, out_features=64, bias=True)
          (fc_k): Linear(in_features=64, out_features=64, bias=True)
          (fc_v): Linear(in_features=64, out_features=64, bias=True)
          (fc_o): Linear(in_features=64, out_features=64, bias=True)
        )
      )
    )
    (enc): Sequential(
      (0): PMA(
        (mab): MAB(
          (fc_q)

In [86]:
# actual metrics
results_settrans, _ = test_model_custom(model_settrans, test_dataloader, compare_solved_sort_unique, print_every=999999, x_name=0, y_name=1)
print('Result: {:.4f}'.format(results_settrans))

Result: 0.0090


In [87]:
# show rank correlations only with mask on
print('K-Tau: {:.4f}, % valid: {}'.format(*get_batch_rank_correlation_and_perc_valid(test_dataloader, model_settrans, get_single_kendall_tau, print_every=999999)))
print('S-Rho: {:.4f}, % valid: {}'.format(*get_batch_rank_correlation_and_perc_valid(test_dataloader, model_settrans, get_single_spearman_rho, print_every=999999)))

K-Tau: 0.2983, % valid: 100.0
S-Rho: 0.4430, % valid: 100.0


#### Sacred Experiment | Model Set Transformer
We'll want to track the dataset, model params and the result of tests.

In [88]:
# experiment config
current_tested_model = model_settrans
current_optimizer = optimizer_settrans
current_last_loss = last_loss_settrans

config_update = {
    'model': current_tested_model.__class__.__name__,
    'final_training_loss': round(current_last_loss.item(), 10),
    'optimizer': current_optimizer.__class__.__name__,
    'cfg': config,
}

# run the experiment, with updated config
run_info = ex.run(config_updates=config_update)

INFO - real_catalogs_PROCAT_mask_True - Running command 'run_model_tests'
INFO - real_catalogs_PROCAT_mask_True - Started run with ID "92"


Model:  <class 'procat_models.SetTransformerPointerCatalogNetwork'>
Result: 0.0090
K-Tau: 0.2983, perc_valid: 100.0


INFO - real_catalogs_PROCAT_mask_True - Result: 0.00902
INFO - real_catalogs_PROCAT_mask_True - Completed after 0:03:18


S-Rho: 0.4430, perc_valid: 100.0


#### Show predictions
See what it predicts.

##### Specific catalog prediction, with offer text features

In [89]:
# choose a catalog
chosen_catalog_id = '0003leq'

Use premade function to nicely display a single prediction.

In [90]:
# show predicted
# if the model is not permutation invariant, it will predict sth else
# for every random permutation of the source catalog
show_predicted_catalog(
    catalog_id='0003leq',
    a_model=model_settrans,
    catalogs_dataframe=df_catalogs,
    offers_dataframe=df)

	3cf8qqfY h: Oreo eller prince chokoladekiks 154-240 , d: 154-240 gram. Flere varianter
	5dbdl0fY h: Axe deodoranter Flere varianter. 150 ml., d: Flere varianter. 150 ml. • Spray
	db9dGqfY h: God morgen juice Appelsin/blodgrape Appe, d: Appelsin/blodgrape Appelsin. LITER. FIND
	22edWqfY h: Flensted kartoffelgratin med fløde 500 g, d: med fløde 500 gram.
	aeberqfY h: Yoggi yoghurt 1000 gram. . Flere variant, d: 1000 gram. . Flere varianter
	08cfm0fY h: Danpo hakket kylling 450 gram, d: 450 gram
	539csqfY h: Mou kødboller 650 gram, d: 650 gram
	ac74V0fY h: Hårspray ELNETT. • Extra kraftig • Stron, d: ELNETT. • Extra kraftig • Strong hold. 4
	2502x0fY h: Leeuwenkuil TA' • Chenin Blanc • Shiraz , d: TA' • Chenin Blanc • Shiraz • Lion's Lai
	907ajqfY h: Æbler 2 kg. pink lady. vildt billigt, d: 2 kg. pink lady. vildt billigt
	7d7a10fY h: Nakkefilet DANSK. 1/2 KG. Skæres til nak, d: DANSK. 1/2 KG. Skæres til nakkekotelette
	f1a9z0fY h: Bki kaffe 500 gram. • Classic Gold • ABC, d: 500 gram. • 

### Persist & reload the model

In [91]:
torch.save(model_settrans, config['model_path'])

In [92]:
# reload and check
model_settrans = torch.load(config['model_path'])
model_settrans.eval()
print('K-Tau: {:.4f}, % valid: {}'.format(*get_batch_rank_correlation_and_perc_valid(test_dataloader, model_settrans, get_single_kendall_tau, print_every=999999)))
print('S-Rho: {:.4f}, % valid: {}'.format(*get_batch_rank_correlation_and_perc_valid(test_dataloader, model_settrans, get_single_spearman_rho, print_every=999999)))

K-Tau: 0.2983, % valid: 100.0
S-Rho: 0.4430, % valid: 100.0


## Model Custom

In [93]:
from procat_models import CustomAttentionSetLayer, CustomAttentionSetEmbedder, CustomAttentionPointerCatalogNetwork

In [94]:
# instantiate the model
model_custom = CustomAttentionPointerCatalogNetwork(
    word_embedding_dim=config['ptr_word_emb_dim'],
    offer_embedding_dim=config['ptr_offer_emb_dim'],
    hidden_dim=config['ptr_hid_dim'],
    offer_rnn_layers=config['ptr_offer_rnn_layers'],
    catalog_rnn_layers=config['ptr_catalog_rnn_layers'],
    dropout_offers=config['ptr_dropout_offers'],
    dropout_catalogs=config['ptr_dropout_catalogs'],
    bidir_offers=config['ptr_bidir_offers'],  # not handled 2021 04
    bidir_catalogs=config['ptr_bidir_catalogs'],
    masking=config['ptr_mask'],
    vocab_size=len(word2idx),
)

model_custom.float()

CustomAttentionPointerCatalogNetwork(
  (word_embedding): Embedding(300005, 32)
  (offer_embedding): PointerOfferEmbedder(
    (offer_rnn): GRU(32, 64, num_layers=2, batch_first=True, dropout=0.05)
  )
  (set_embedding): CustomAttentionSetEmbedder(
    (l1): CustomAttentionSetLayer(
      (e): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=64, bias=True)
        (3): ReLU()
      )
      (s): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=1, bias=True)
        (3): ReLU()
      )
      (a): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=64, bias=True)
        (3): ReLU()
      )
    )
    (l2): CustomAttentionSetLayer(
      (e): Sequential(
        (0): Linear(in_features=64, out_features=64, bia

In [95]:
# how many params
count_params(model_custom)

The model has 9,860,260 trainable parameters


In [96]:
# 2-elem batch
batch_x = X_train_torch[:2]

In [97]:
# predict
_ = model_custom(batch_x)
print(_[0].size(), _[1].size())

torch.Size([2, 200, 200]) torch.Size([2, 200])


In [98]:
print(_[1].type())

torch.LongTensor


In [99]:
# loss
CCE = torch.nn.CrossEntropyLoss()

In [100]:
# optimizer
optimizer_custom = optim.Adam(
    filter(lambda p: p.requires_grad, model_custom.parameters()),
    lr=config['learning_rate'])

### Model Training | Custom

In [101]:
# train
last_loss_custom = train_epochs(
    model_custom,
    optimizer_custom,
    CCE,
    train_dataloader,
    config['num_epochs'],
    allow_gpu=True,
    x_name=0,
    y_name=1)

CUDA available, using GPU:
Quadro P6000


Epoch 5 / 5: 100%|██████████| 139/139 [03:14<00:00,  1.40s/ batches, avg_loss=5.29841]


In [102]:
last_loss_custom.item()

5.298410892486572

### Model Testing | Custom
Put the model in eval() mode first (prevents dropout and such from predicting sth else from the same permuted x).

In [103]:
model_custom.eval()  # if we had dropout or batchnorm, we must do this

CustomAttentionPointerCatalogNetwork(
  (word_embedding): Embedding(300005, 32)
  (offer_embedding): PointerOfferEmbedder(
    (offer_rnn): GRU(32, 64, num_layers=2, batch_first=True, dropout=0.05)
  )
  (set_embedding): CustomAttentionSetEmbedder(
    (l1): CustomAttentionSetLayer(
      (e): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=64, bias=True)
        (3): ReLU()
      )
      (s): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=1, bias=True)
        (3): ReLU()
      )
      (a): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=64, bias=True)
        (3): ReLU()
      )
    )
    (l2): CustomAttentionSetLayer(
      (e): Sequential(
        (0): Linear(in_features=64, out_features=64, bia

#### Sacred Experiment | Model Custom
We'll want to track the dataset, model params and the result of tests.

In [104]:
# experiment config
current_tested_model = model_custom
current_optimizer = optimizer_custom
current_last_loss = last_loss_custom

config_update = {
    'model': current_tested_model.__class__.__name__,
    'final_training_loss': round(current_last_loss.item(), 10),
    'optimizer': current_optimizer.__class__.__name__,
    'cfg': config,
}

# run the experiment, with updated config
run_info = ex.run(config_updates=config_update)

INFO - real_catalogs_PROCAT_mask_True - Running command 'run_model_tests'
INFO - real_catalogs_PROCAT_mask_True - Started run with ID "93"


Model:  <class 'procat_models.CustomAttentionPointerCatalogNetwork'>
Result: 0.0050
K-Tau: 0.0133, perc_valid: 100.0


INFO - real_catalogs_PROCAT_mask_True - Result: 0.00497
INFO - real_catalogs_PROCAT_mask_True - Completed after 0:03:15


S-Rho: 0.0174, perc_valid: 100.0


#### Show predictions
See what it predicts.

##### Specific catalog prediction, with offer text features

In [105]:
# choose a catalog
chosen_catalog_id = '0003leq'

In [106]:
# show correct
show_correct_catalog(
    catalog_id=chosen_catalog_id,
    catalogs_dataframe=df_catalogs,
    offers_dataframe=df)

	fed3AqfY h: Påskebryg 33 cl. KYLLE - KYLLE. 3 33. + , d: 33 cl. KYLLE - KYLLE. 3 33. + PANT. Sælg
	799dzqfY h: Frisk dansk mælk PR. LITER. Max. 24 time, d: PR. LITER. Max. 24 timer fra gård til bu
	b8a3DqfY h: Sukkervafler BELGISKE. 550 gram, d: BELGISKE. 550 gram
	a4c88qfY h: Hatting fransk hotdog  gigant burgerboll, d: 4-6 stk. •
	3cf8qqfY h: Oreo eller prince chokoladekiks 154-240 , d: 154-240 gram. Flere varianter
	88a5UqfY h: Knækbrød LEKSANDS. Flere varianter 200 g, d: LEKSANDS. Flere varianter 200 gram
	6d6e0qfY h: Carletti 73,5 gram. • Lys • Mørk. PÅLÆGS, d: 73,5 gram. • Lys • Mørk. PÅLÆGSCHOKOLADE
	ef59YqfY h: Corny müslibars 6 stk. • Nuts, d: 6 stk. • Nuts
	ab38CqfY h: BKI KAFFE  • Extra. 400 gram. Ved køb af, d: • Extra. 400 gram. Ved køb af mere end 1
	0cbffqfY h: Pålækker 80-90 gram. - Classic - Tomat. , d: 80-90 gram. - Classic - Tomat. VILDT BIL
	0a4eEqfY h: Havregryn Ved køb af mere end 6 poser, e, d: Ved køb af mere end 6 poser, er prisen h
------------------------
	0

In [107]:
# show predicted
# if the model is not permutation invariant, it will predict sth else
# for every random permutation of the source catalog
show_predicted_catalog(
    catalog_id='0003leq',
    a_model=model_custom,
    catalogs_dataframe=df_catalogs,
    offers_dataframe=df)

	5cb900fY h: Skinkestege 700-800 gram. • Orientalsk •, d: 700-800 gram. • Orientalsk • Mexico
	0ef3QqfY h: Kyllingelår KUN 11,66 KR. PR. KG DET ER , d: KUN 11,66 KR. PR. KG DET ER BILLIGT. HEL
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	c90d30fY h: Steaks Ca. 1000 gram. AF UNGKVÆG. Små fi, d: Ca. 1000 gram. AF UNGKVÆG. Små fine hver
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	431baqfY h: Toast 200-290 gram. • Pariser toast • Co, d: 200-290 gram. • Pariser toast • Cowboy t
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	11bbPqfY h: Blomme tomater 500 gram, d: 500 gram
	a4c88qfY h: Hatting fransk hotdog  gigant burgerboll, d: 4-6 stk. •
	a5caD0fY h: Jægerkoteletter Ca. 400 gram. Marinerede, d: Ca. 400 gram. Marinerede koteletter med 
	 ?NOT_REAL_OFFER?
	b553q0fY h: Helt kalkunbryst Ca. 800 gram, d: Ca. 800 gram
	3956N0fY h: Harpic wc-rens Flere varianter 750 ml. K, d: Flere varianter 750 ml. KÆMPE PARTI
	283c80fY h: Nescafé gold STORT GLAS. • Instant kaffe, d: STORT GL

	ef59YqfY h: Corny müslibars 6 stk. • Nuts, d: 6 stk. • Nuts
	18cdFqfY h: Hønskød i tern 400 gram. Kogt og klar ti, d: 400 gram. Kogt og klar til brug i f.eks.
	d8f9oqfY h: Margarine •. 500 gram. BINE. BILLIGST PÅ, d: •. 500 gram. BINE. BILLIGST PÅ HELE INDK
	44a2IqfY h: Tulip •. 300-350 gram. • Baconpostej • K, d: •. 300-350 gram. • Baconpostej • Krydder
	 ?NOT_REAL_OFFER?
	4815F0fY h: Sodavand 150 cl. Flere varianter. + PANT, d: 150 cl. Flere varianter. + PANT. 3. • Ni
	799dzqfY h: Frisk dansk mælk PR. LITER. Max. 24 time, d: PR. LITER. Max. 24 timer fra gård til bu
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	d638dqfY h: Frigodan ISKOLD PRIS. • Broccoli • Fine , d: ISKOLD PRIS. • Broccoli • Fine ærter • M
	36f3y0fY h: Danpo • Frikadeller • Medister. 400 gram, d: • Frikadeller • Medister. 400 gram
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	db9dGqfY h: God morgen juice Appelsin/blodgrape Appe, d: Appelsin/blodgrape Appelsin. LITER. FIND
	 ?NOT_REAL_OFFER?
	